In [34]:
import matplotlib.pyplot as plt
import os
import numpy as np
import subprocess

In [35]:
def mean_squared_error(y_hat, y):
    return np.sum((y_hat - y)**2)/y.size

In [36]:
y = np.array([9,8,7,6])
y_hat1 = np.array([1.2, 1.9, 2.9,  4.2]) 
mean_squared_error(y_hat1, y)

29.524999999999995

In [37]:
y_hat2 = np.array([2.2, 0.9, 2.9,  5.2]) 
mean_squared_error(y_hat2, y)

28.525

In [38]:
def cross_entropy_error(y_hat, y):
    return -np.sum(y*np.log(y_hat + 1e-7))

In [39]:
y = np.array([0, 1, 0, 0, 0])

In [40]:
y_hat1 = np.array([0.1, 0.7, 0.1, 0.1, 0])

In [41]:
cross_entropy_error(y_hat1, y)

0.3566748010815999

In [ ]:
import mnist_data as mnist

my_mnist = mnist.MnistData()

ModuleNotFoundError: No module named 'mnist'

In [24]:
# Define the base directory for your handwriting images
base_dir = 'Custom MNIST Samples'

correctCount = 0
wrongCount = 0
totalImages = 0

# Loop through each digit and each image, running module5-3.py
for digit in range(10):
    digit_dir = os.path.join(base_dir, f'Digit {digit}')
    
    if os.path.isdir(digit_dir):
        for filename in os.listdir(digit_dir):
            if filename.endswith('.jpg'):
                image_path = filename
                # print(image_path)
                totalImages+=1
                result = subprocess.run(["python","module6.py",image_path,str(filename)[0]],capture_output=True,text=True)
                output = result.stdout.strip()
                # print(output)
                if "Success" in output:
                    correctCount+=1
                else:
                    wrongCount+=1
            # if totalImages==10: break
        # if totalImages==10: break

print(f'In total there are {totalImages} number of images out of which there are {correctCount} images predicted correctly and there are {wrongCount} images predicted wrongly.')

In total there are 50 number of images out of which there are 38 images predicted correctly and there are 12 images predicted wrongly.
